In [19]:
import os
import tensorflow as tf
from tensorflow.keras.models import load_model
from UNetRMultiClass import logger
import cv2
import numpy as np
from patchify import patchify
import onnx
import onnxruntime as ort

In [5]:
# %pwd

'd:\\Machine Learning Projects\\Unet-R Full-stack\\ML_model'

In [29]:
class PredictionPipeline:
    def __init__(self,filename="testing"):
        self.filename = filename
        self.rgb_codes = [
        [0, 0, 0], [0, 153, 255], [102, 255, 153], [0, 204, 153],
        [255, 255, 102], [255, 255, 204], [255, 153, 0], [255, 102, 255],
        [102, 0, 51], [255, 204, 255], [255, 0, 102]
        ]

        self.classes  = [
        "background", "skin", "left eyebrow", "right eyebrow",
        "left eye", "right eye", "nose", "upper lip", "inner mouth",
        "lower lip", "hair"
        ]
        

    def grayscale_to_rgb(self, mask, rgb_codes):
        h, w = mask.shape[0], mask.shape[1]
        mask = mask.astype(np.int32)
        output = []
        
        enum = enumerate(mask.flatten())
        
        for i, pixel in enum:
            output.append(rgb_codes[pixel])

        output = np.reshape(output, (h, w, 3))
        
        return output

    def save_results(self, image_x, pred, save_image_path):

        pred = np.expand_dims(pred, axis=-1)
        pred = self.grayscale_to_rgb(pred, self.rgb_codes)

        line = np.ones((image_x.shape[0], 10, 3)) * 255

        cat_images = np.concatenate([image_x, line, pred], axis=1)
        
        cv2.imwrite(save_image_path, cat_images)


    
    def predict(self):
        cf = {}
        cf["image_size"] = 256
        cf["num_classes"] = 11
        cf["num_channels"] = 3
        cf["num_layers"] = 12
        cf["hidden_dim"] = 128
        cf["mlp_dim"] = 32
        cf["num_heads"] = 6
        cf["dropout_rate"] = 0.1
        cf["patch_size"] = 16
        cf["num_patches"] = (cf["image_size"]**2)//(cf["patch_size"]**2)
        cf["flat_patches_shape"] = (
            cf["num_patches"],
            cf["patch_size"]*cf["patch_size"]*cf["num_channels"]
        )
        
        onnx_model_path = "artifacts/training/compatible_model.onnx"
        print("model_path: ", onnx_model_path)
        o_model = onnx.load(onnx_model_path)
        session = ort.InferenceSession(onnx_model_path)
        input_name = session.get_inputs()[0].name

        
        image_name = self.filename
        display_name = image_name.split('/')[-1].split('.')[0]
        logger.info(f"input_image name is: {display_name}")

        input_img = cv2.imread(image_name, cv2.IMREAD_COLOR)
        input_img = cv2.resize(input_img, (cf["image_size"], cf["image_size"]))
        norm_input_img = input_img / 255.0

        patch_shape = (cf["patch_size"], cf["patch_size"], cf["num_channels"])
        patches = patchify(norm_input_img, patch_shape, cf["patch_size"])
        patches = np.reshape(patches, cf["flat_patches_shape"])
        patches = patches.astype(np.float32) #[...]
        patches = np.expand_dims(patches, axis=0) # [1, ...]

        """ Prediction """

        input_dict = {input_name: patches}
        output_name = session.get_outputs()[0].name
        
        outputs = session.run([output_name], input_dict)
        pred_1 = np.argmax(outputs, axis=-1) ## [0.1, 0.2, 0.1, 0.6] -> 3
        pred_1 = pred_1.astype(np.int32)
        pred_1 = np.reshape(pred_1, (256, 256))
        
        save_image_path = f"outputs/predict/{display_name}.png"
        self.save_results(input_img, pred_1, save_image_path)
        

In [31]:
# fname = os.path.join("artifacts", "LaPa", "test", "images", "2569520_1.jpg")
fname = f"artifacts/LaPa/test/images/2569520_1.jpg"
pred = PredictionPipeline(filename=fname)
pred.predict()

model_path:  artifacts/training/compatible_model.onnx
[2024-05-07 20:16:38,623: INFO: 411376372: input_image name is: 2569520_1]


In [22]:
os.path.exists(fname)

True